In [6]:
%aiida

Loaded AiiDA DB environment - profile name: bmourino.

In [7]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import glob
import fileinput
import sys
import csv
import math
import matplotlib.pyplot as plt

from aiida import load_profile
from aiida.tools import get_kpoints_path
from aiida.orm.querybuilder import QueryBuilder
from aiida.plugins import CalculationFactory, WorkflowFactory
from aiida.orm import StructureData, Dict, load_node, CalcFunctionNode, CalcJobNode, WorkChainNode, BandsData, CifData

import macrodensity as md
from macrodensity.cp2k_tools_maria import read_cube_density
from macrodensity.cp2k_tools_maria import read_cell
from macrodensity.cp2k_tools_maria import cell_to_cellpar
from macrodensity.cp2k_tools_maria import read_geo
from macrodensity.cp2k_tools_maria import test_point

from manage_crystal.utils import parse_and_write
from manage_crystal.file_parser import parse_from_filepath
from manage_crystal.file_writer import write_to_filepath

from pymatgen.io.xyz import XYZ
from pymatgen.io.cif import CifParser,CifWriter

AU2EV = 27.211399

In [70]:
def get_cellopt_retrieved_folder(wc_pk): 
    wc = load_node(wc_pk)
    for des in wc.called_descendants:
        if des.label == 'CELL_OPT': 
            folder = folder = des.get_retrieved_node()
            folder_path = folder._repository._get_base_folder().abspath
            return folder_path
    return None

def get_cube_path_smb(wc_pk):
    cubepath = get_cellopt_retrieved_folder(wc_pk)
    cubesmb = '/home/beatriz/nas/conv_cofs_mofs_3/node' + cubepath.split('node')[-1] #can be better - I can make it case-based so it works for all of the backed up folders
    return cubesmb

def get_opt_cell(wc_pk):
    path = get_cellopt_retrieved_folder(wc_pk)
    
def replace_cell_xyz(file):
    for line in fileinput.input(file,inplace=1):
        if "CELL" in line:
            line = line.replace("CELL","xyz")
        sys.stdout.write(line)

def get_band_alignment(hartreecube,opt,cell):
    input_file = hartreecube
    geo_file = opt
    cell_file = cell
    cube_size = [3,3,3] 
    
    pot, NGX, NGY, NGZ, Lattice = read_cube_density(input_file)
    vector_a,vector_b,vector_c,av,bv,cv = md.matrix_2_abc(Lattice)
    resolution_x = vector_a/NGX
    resolution_y = vector_b/NGY
    resolution_z = vector_c/NGZ
    c1=0.5/vector_a
    c2=0.5/vector_b
    c3=0.5/vector_c
    grid_pot = md.density_grid_cube(pot,NGX,NGY,NGZ)
    
    cube = cube_size
    travelled = [0,0,0]

    cell = read_cell(cell_file)
    params = cell_to_cellpar(cell,radians=False)

    atom_type,coord,num_atoms = read_geo(geo_file)

    thr=2.0

    cube_var_l=1000
    cube_potential_l=0
    d_l=0
    f_l=0
    g_l=0
    logical=0
    pot = []
    var = []
    for d in np.arange(0.0,1.0,c1):
        for f in np.arange(0.0,1.0,c2):
            for g in np.arange(0.0,1.0,c3):
                cube_origin = [d,f,g]
                logical = test_point(cube_origin,coord,params,num_atoms,thr)
                #print(d,f,g,logical)
                if logical == 1:
                    cube_potential, cube_var = md.cube_potential(cube_origin,travelled,cube,grid_pot,NGX,NGY,NGZ)
                    if cube_var < cube_var_l:
                        cube_var_l=cube_var
                        cube_potential_l=cube_potential
                        d_l=d
                        f_l=f
                        g_l=g
                        print(" %4.1f %5.2f   %5.2f   %5.2f    %10.4f   %10.6f"%(thr,d_l,f_l,g_l,cube_potential_l,cube_var_l))
                        pot.append(cube_potential_l)
                        var.append(cube_var_l)
    return pot,var
              
def get_cube_ba(pk,relaxedstr):
    cubepath = get_cube_path_smb(pk)
    hartree_cube = glob.glob(os.path.join(cubepath, "*hartree*.cube"))
    
    celloptfolder = get_cellopt_retrieved_folder(pk)
    cellpath = celloptfolder + "/cell.csv"
    with open(cellpath, "w") as f:
        wr = csv.writer(f, delimiter=" ")
        wr.writerows(relaxedstr.cell)    
    
    optpath = relaxedstr.export(celloptfolder + '/opt.xyz')
    
    lines = []
    #reading lines
    with open(optpath[0], 'r') as f:
        #going through each line and storing in list
        lines = f.readlines()

    with open(optpath[0], 'w') as f:
        for line in lines:
            if line.startswith("Lattice"):
                continue
            else:
                f.write(line)    
    
    lines = []
    #reading lines
    with open(optpath[0], 'r') as f:
        #going through each line and storing in list
        lines = f.readlines()

    numat = int(lines[0].split("\n")[0])
    numat = str(numat-1) + "\n"
    lines[0] = numat
    lines[1] = "xyz" + "\n"

    with open(optpath[0], 'w') as f:
        for line in lines:
            f.write(line)
    
    optxyz = optpath[0]
    cellcsv = cellpath
    pot,var = get_band_alignment(hartree_cube[0],optxyz,cellcsv)
    
    return pot[-1],var[-1]

def get_homo_lumos(wc_pk, name, uuid): 
    wc = load_node(wc_pk)
    for i in range(5):
        wcn = wc.called[i]
        if wcn.label == 'stage_1_settings_0':
            wcnn = wcn.called[0]
            if wcnn.label == 'CELL_OPT':
                folder = wcnn.get_retrieved_node()
                folder_path = folder._repository._get_base_folder().abspath
                aiidaout = folder_path + "/aiida.out"
                with open(aiidaout, 'r') as fh:
                    lines = []
                    for line in fh.readlines():
                        lines.append(line)
                        if 'Fermi' in line:
                            homo_e = float(lines[-2].split()[-1])*AU2EV #is this right?
                        if "HOMO - LUMO gap" in line:
                            bandgap = float(line.split()[-1])
                            lumo_e = float(lines[-3].split()[-1])*AU2EV
    return {
        'pk': wc_pk,
        'homo_energy': homo_e,
        'lumo_energy': lumo_e,
        'bandgap': bandgap,
        'name':name,
        'path':uuid
    }

In [71]:
Cp2kCalculation = CalculationFactory('cp2k')
Cp2kBaseWorkChain = WorkflowFactory('cp2k.base')
Cp2kMultistageWorkChain = WorkflowFactory('lsmo.cp2k_multistage')
Cp2kBandsWorkChain = WorkflowFactory('photocat_workchains.bandstructure')
Cp2kPhotoCatWorkChain = WorkflowFactory('photocat_workchains.cp2k_photocat')

In [72]:
qb = QueryBuilder()
qb.append(Group, filters={'label': 'ocofs-conv2'}, tag='group')
qb.append(Cp2kPhotoCatWorkChain, with_group='group', tag='photocat') #specifying the workchain I want to inspect; if I don't put the uuid it won't be in the dict
qb.append(CalcFunctionNode, filters={'label': {'==': 'get_structure_from_cif'}}, tag='get_st', with_incoming='photocat') #this is to be able to append CifData (it's important because their labels are the structure name)
qb.append(CifData, project=['label'], with_outgoing='get_st', tag='structure_label') #appending CifData to later get the structure name
qb.append(Cp2kMultistageWorkChain, with_incoming='photocat', filters={'and': [{'attributes.process_state':{'==':'finished'}}]}, project='uuid', tag='ms_wc') #appending multistage (all of them, including the single point)
qb.append(Cp2kBaseWorkChain, with_incoming='ms_wc', filters={'label': {'==':'stage_1_settings_0'}}, project='uuid', tag='cellopt') #appending multistage (all of them, including the single point)
qb.append(StructureData, project=['uuid'], with_outgoing='cellopt') #structures used for ms_wcqb.append(Dict, with_incoming='ms_wc', project=['uuid'], filters={'attributes': {'has_key': 'natoms'}}) #still don't know what this is for
qb.append(StructureData, project=['uuid'], with_outgoing='ms_wc') #structures used for ms_wc
# qb.append(Dict, with_incoming='ms_wc', project=['uuid'], filters={'attributes': {'has_key': 'natoms'}}) #still don't know what this is for
qb.append(StructureData, with_incoming='ms_wc', project=['uuid'], tag='relaxed') #relaxed structure; will filter ms_wc that are not single point (not appending anymore the single point ones, since the structures here have to be generated by a ms_wc - incoming - and a single point ms_wc won't generate structures)
qb.append(CalcFunctionNode, filters={'label': {'==': 'structure_with_pbc'}}, project=['uuid'], with_incoming='photocat', tag='get_pbc_calcfunction') #this is to be able to append primitive structures
qb.append(StructureData, with_incoming='get_pbc_calcfunction', project=['uuid'], tag='structure_with_pbc') #appending primitive structures so I can use them for plotting bands
qb.append(Cp2kBandsWorkChain, tag='bands_wc', project=['uuid']) #this is apending the bandst workchain
qb.append(BandsData, project=['uuid'], with_incoming='bands_wc') #this is appending the data from bands_wc, i.e., kpoints, bands, labels

In [73]:
qb.all()

[['20290N2_ddec.cif',
  '9233bdb3-0ebe-47fa-ab57-fc107073df53',
  'b7c32518-10b2-4074-b573-7c7a1a489654',
  '83dc6c76-921e-41ef-962d-2f8452b23c62',
  '65175470-6f9d-4aff-a4b0-dd7b4a826529',
  '990c52a3-1fd8-4eb5-b9b7-468252c514fc',
  '6640f8a6-b404-4719-b91e-d0bae206a4a9',
  'b21191f6-22ea-4312-b08d-56d900224d6c',
  'ddd2ca2f-dfa6-4872-a4a4-a6a219cfd007',
  'cc5033ee-9227-4ff0-b533-9b0e83031850'],
 ['20290N2_ddec.cif',
  '9233bdb3-0ebe-47fa-ab57-fc107073df53',
  'b7c32518-10b2-4074-b573-7c7a1a489654',
  '83dc6c76-921e-41ef-962d-2f8452b23c62',
  '65175470-6f9d-4aff-a4b0-dd7b4a826529',
  '990c52a3-1fd8-4eb5-b9b7-468252c514fc',
  '6640f8a6-b404-4719-b91e-d0bae206a4a9',
  'b21191f6-22ea-4312-b08d-56d900224d6c',
  'ddd2ca2f-dfa6-4872-a4a4-a6a219cfd007',
  'cc5033ee-9227-4ff0-b533-9b0e83031850'],
 ['19482N2_ddec.cif',
  'f508d5ee-0d42-4c41-bd05-2829c3bdbab8',
  '3e9f7ca0-699c-4e9f-869a-f86c4862d087',
  'e33c9817-0b1b-462b-9339-4997af598a67',
  '922b15cc-8343-4272-b921-77f234348a76',
  '6ff94

In [74]:
qb_results_dict = {}
qb_results  = []
names = []
## eliminate duplicates:
for result in qb.all():

    qb_results_dict[result[0]] = result
    if result[0] in names:
        continue
    else:
        names.append(result[0])
        qb_results.append(result)
len(qb_results)

27

In [75]:
label = []
for i,j in enumerate(qb_results[0:]):
    label.append(j[0])
print(label)

['20290N2_ddec.cif', '19482N2_ddec.cif', '19103N2_ddec.cif', '21030N2_ddec.cif', '15090N2_ddec.cif', '20542N2_ddec.cif', '15242N2_ddec.cif', '16260N2_ddec.cif', '16032N2_ddec.cif', '19071N2_ddec.cif', '17155N2_ddec.cif', '16172N2_ddec.cif', '20161N2_ddec.cif', '19483N2_ddec.cif', '20163N2_ddec.cif', '19220N2_ddec.cif', '16510N2_ddec.cif', '2016bN2_ddec.cif', '20167N2_ddec.cif', '19341N2_ddec.cif', '20510N2_ddec.cif', '11001N2_ddec.cif', '17050N2_ddec.cif', '09000N3_ddec.cif', '20000N2_ddec.cif', '19560N2_ddec.cif', '18140N2_ddec.cif']


In [76]:
all_pot = []
homo_lumos = []
for result in qb_results[0:]:
    try: 
        name = result[0]
        print(name)
        ms = load_node(result[1])
        relaxedstr = load_node(result[4])
        pot,var = get_cube_ba(ms.pk,relaxedstr)
        result = {
            'label': name, 
            'potential': pot,
            'var': var,
        }
        print(result)
        all_pot.append(result)

        results = get_homo_lumos(ms.pk,name,ms.uuid)
        homo_lumos.append(results)
    except Exception as e:
        print(e)

20290N2_ddec.cif
  2.0  0.00    0.00    0.00        3.4144     0.001705
  2.0  0.00    0.00    0.05        3.4109     0.000406
  2.0  0.00    0.05    0.00        3.3824     0.000267
  2.0  0.00    0.59    0.86        3.2922     0.000198
  2.0  0.11    0.16    0.59        3.8403     0.000172
  2.0  0.14    0.16    0.59        3.8433     0.000126
{'label': '20290N2_ddec.cif', 'potential': 3.843297681576296, 'var': 0.00012604116160755307}
19482N2_ddec.cif
  2.0  0.00    0.00    0.00        0.5294    11.856719
  2.0  0.00    0.00    0.09        2.5299     0.022884
  2.0  0.00    0.03    0.50        2.6799     0.013879
  2.0  0.00    0.05    0.50        2.6410     0.005707
  2.0  0.00    0.06    0.50        2.6002     0.001589
  2.0  0.00    0.73    0.18        2.5757     0.001479
  2.0  0.05    0.45    0.14        2.5871     0.000894
  2.0  0.30    0.45    0.14        2.5835     0.000764
  2.0  0.92    0.44    0.14        2.5660     0.000691
{'label': '19482N2_ddec.cif', 'potential': 2.566

In [54]:
df = pd.DataFrame(homo_lumos)
pot = pd.DataFrame(all_pot)
df = df.assign(potential=pot['potential'])
df = df.assign(var=pot['var'])
df['homo_align'] = df['homo_energy'] - df['potential']
df['lumo_align'] = df['lumo_energy'] - df['potential']
df.to_csv('../output_data/oconv2/balign/ba.csv',index=False)

In [69]:
#remove opt.xyz
for result in qb_results[0:]:
    name = result[0]
    print(name)
    ms = load_node(result[1])
    path = get_cellopt_retrieved_folder(ms.pk)
    try:
        optpath = path + "/opt.xyz"
        print(optpath)
        os.unlink(optpath)
    except:
        print("No opt.xyz file to remove")

20290N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/85/1e/182e-967c-460b-9398-0c39f98555f2/path/opt.xyz
19482N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/4f/14/29c3-a568-41ae-9da6-6415d9e48ff9/path/opt.xyz
19103N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/c3/d5/7ae7-120a-4658-be96-7be2e657dff3/path/opt.xyz
21030N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/90/02/2283-ab77-42b0-9f34-fa62370ef675/path/opt.xyz
15090N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/90/c5/ce6d-8801-41eb-801b-c5ecc0ec2c36/path/opt.xyz
20542N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/a4/b7/bdc1-a7c6-4822-848b-e456e9d8da95/path/opt.xyz
15242N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/4b/81/7fd4-3c08-4f1d-9c96-13d2e1d7be69/path/opt.xyz
16260N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/28/95/950a-cb77-43b7-a074-fa76148872aa/path/

In [18]:
# df = pd.DataFrame(homo_lumos)
# pot = pd.DataFrame(all_pot)
# df = df.assign(potential=pot['potential'])
# df = df.assign(var=pot['var'])
# df['homo_align'] = df['homo_energy'] - df['potential']
# df['lumo_align'] = df['lumo_energy'] - df['potential']
# df.to_csv('../output_data/obatch36/ba.csv',index=False)

In [11]:
# for result in qb_results[0:]:
#     name = result[0]
#     print(name)
#     ms = load_node(result[1])
#     path = get_cellopt_retrieved_folder(ms.pk)
#     print(path)